# LIFE E2E — Quick Demo

This notebook reproduces the headline results from Huarcaya (2026) in ~30 seconds
using a reduced Monte Carlo sample (N=1,000 vs 10⁵ in the paper).

**Paper:** _Analytical Throughput, Null Depth, and Surface Tolerance Budget_
_for the LIFE Nulling Interferometer Combiner_ — A&A (in prep.)

**DOI:** [10.5281/zenodo.18716470](https://doi.org/10.5281/zenodo.18716470)

In [ ]:
import sys, os
import numpy as np
import matplotlib.pyplot as plt
import warnings

# Add module path
sys.path.insert(0, os.path.join('..', 'src', 'life_e2e'))
warnings.filterwarnings('ignore', message='.*CaF.*Sellmeier.*')

print('Imports OK')

## 1. Material Properties

Wavelength-dependent optical constants for the LIFE combiner materials.

In [ ]:
from material_properties import (gold_reflectivity, caf2_sellmeier,
                                  znse_sellmeier, detector_qe)

lam = np.linspace(4, 20, 200)

fig, axes = plt.subplots(1, 3, figsize=(14, 4))

# Gold reflectivity
axes[0].plot(lam, gold_reflectivity(lam, 'flight') * 100, 'goldenrod', lw=2)
axes[0].set(xlabel='λ [µm]', ylabel='R [%]', title='Gold reflectivity (flight)',
            ylim=(96, 100))

# Refractive indices
axes[1].plot(lam, caf2_sellmeier(lam), 'C0', lw=2, label='CaF₂')
axes[1].plot(lam, znse_sellmeier(lam), 'C1', lw=2, label='ZnSe')
axes[1].set(xlabel='λ [µm]', ylabel='n', title='Refractive index')
axes[1].legend()

# Detector QE
axes[2].plot(lam, detector_qe(lam) * 100, 'C3', lw=2)
axes[2].set(xlabel='λ [µm]', ylabel='QE [%]', title='Si:As BIB detector')

plt.tight_layout()
plt.show()

## 2. Fiber Coupling

Top-hat coupling efficiency η(β) — the 18.5% deficit compared to Gaussian illumination
is a fundamental cost of using flat-top collector beams.

In [ ]:
from fiber_modes import coupling_tophat_analytical, coupling_gaussian_to_gaussian

beta = np.linspace(0.3, 3.0, 500)
eta_tophat = coupling_tophat_analytical(beta)

beta_opt = beta[np.argmax(eta_tophat)]
eta_max = np.max(eta_tophat)

fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(beta, eta_tophat * 100, 'C0', lw=2.5, label='Top-hat → SM fiber')
ax.axhline(100, color='gray', ls='--', alpha=0.5, label='Gaussian → Gaussian (100%)')
ax.axvline(beta_opt, color='C0', ls=':', alpha=0.6)
ax.plot(beta_opt, eta_max * 100, 'ro', ms=10, zorder=5)
ax.annotate(f'  β_opt = {beta_opt:.3f}\n  η₀ = {eta_max*100:.2f}%',
            xy=(beta_opt, eta_max*100), fontsize=12,
            xytext=(beta_opt+0.3, eta_max*100-5))
ax.set(xlabel='β = R_beam / w_f', ylabel='Coupling efficiency [%]',
       title='Fiber coupling: top-hat illumination', xlim=(0.3, 3), ylim=(0, 105))
ax.legend(fontsize=11)
plt.tight_layout()
plt.show()

print(f'Optimal β = {beta_opt:.4f}')
print(f'Maximum coupling = {eta_max*100:.2f}%')
print(f'Coupling deficit vs Gaussian = {(1-eta_max)*100:.2f}%')

## 3. Null Error Budget (Module 3)

Analytical null depth from all error sources across the science band.

In [ ]:
from m3_null_error_propagation import compute_null_budget

wavelengths_m = np.linspace(6, 16, 100) * 1e-6
budget = compute_null_budget(wavelengths_m)

fig, ax = plt.subplots(figsize=(9, 5))
lam_um = wavelengths_m * 1e6

ax.semilogy(lam_um, budget['N_total'], 'k-', lw=2.5, label='Total null depth')
for key, label, ls in [
    ('N_opd_mean', 'OPD mean', '--'),
    ('N_opd_rms', 'OPD rms', '--'),
    ('N_bs_chromatic', 'BS chromatic', '-'),
    ('N_intensity_mean', 'Intensity mean', ':'),
    ('N_intensity_rms', 'Intensity rms', ':'),
]:
    if key in budget:
        ax.semilogy(lam_um, budget[key], ls=ls, lw=1.5, label=label, alpha=0.7)

ax.set(xlabel='Wavelength [µm]', ylabel='Null depth',
       title='Analytical null error budget (Module 3)',
       xlim=(6, 16), ylim=(1e-7, 1e-3))
ax.legend(fontsize=9, loc='upper right')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print('Per-term null budget at 6, 10, 16 µm:')
for key in ['N_total', 'N_bs_chromatic']:
    if key in budget:
        vals = np.interp([6, 10, 16], lam_um, budget[key])
        print(f'  {key:20s}: {vals[0]:.2e}  {vals[1]:.2e}  {vals[2]:.2e}')

## 4. Monte Carlo (Quick Run)

Full end-to-end MC with N=1,000 realisations (~5 seconds).
The paper uses N=100,000 — increase `N_MC` below for publication-quality statistics.

In [ ]:
from monte_carlo import run_monte_carlo
import time

N_MC = 1_000  # Change to 100_000 for paper-quality results

t0 = time.time()
results = run_monte_carlo(
    N_realizations=N_MC,
    wavelengths_um=np.array([6.0, 8.0, 10.0, 12.0, 16.0]),
    seed=42,
    verbose=False,
)
dt = time.time() - t0
print(f'MC completed: {N_MC:,} realisations in {dt:.1f} s')

In [ ]:
wavelengths = results['wavelengths']
null_depths = results['null_depths']
throughputs = results['throughputs']

# --- Summary table ---
print(f'{"λ [µm]":>8}  {"N_mean":>12}  {"N_median":>12}  {"PCE [%]":>10}')
print('-' * 48)
for j, lam in enumerate(wavelengths):
    N_mean = np.mean(null_depths[:, j])
    N_med = np.median(null_depths[:, j])
    pce = np.mean(throughputs[:, j]) * 100
    print(f'{lam:8.0f}  {N_mean:12.2e}  {N_med:12.2e}  {pce:10.2f}')

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Null depth distributions
colors = plt.cm.viridis(np.linspace(0.1, 0.9, len(wavelengths)))
for j, (lam, c) in enumerate(zip(wavelengths, colors)):
    N = null_depths[:, j]
    log_N = np.log10(N)
    axes[0].hist(log_N, bins=50, alpha=0.5, color=c,
                 label=f'{lam:.0f} µm (mean={np.mean(N):.1e})', density=True)

axes[0].set(xlabel='log₁₀(Null depth)', ylabel='Density',
            title=f'MC null depth distributions (N={N_MC:,})')
axes[0].legend(fontsize=8)

# Right: Throughput (PCE)
pce_mean = np.mean(throughputs, axis=0) * 100
pce_std = np.std(throughputs, axis=0) * 100
axes[1].errorbar(wavelengths, pce_mean, yerr=pce_std, fmt='s-', color='C0',
                 capsize=5, lw=2, ms=8)
axes[1].axhspan(3.5, 10, alpha=0.1, color='green', label='Design range (3.5–10%)')
axes[1].set(xlabel='Wavelength [µm]', ylabel='PCE [%]',
            title='Photon conversion efficiency')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Technology Gap Assessment

How far is current (warm-bench) surface quality from the null depth requirement?

In [ ]:
from monte_carlo import get_null_requirements

req = get_null_requirements()

print(f'{"λ [µm]":>8}  {"N_MC":>12}  {"N_req":>12}  {"Gap":>8}  {"σ improve":>12}')
print('-' * 58)
for j, lam in enumerate(wavelengths):
    N_mc = np.mean(null_depths[:, j])
    N_req_val = req['birbacher'].get(lam, 1e-5)
    gap = N_mc / N_req_val
    sigma_improve = gap ** (1/4)  # quartic scaling
    status = '✓ PASS' if gap <= 1 else f'{gap:.0f}×'
    print(f'{lam:8.0f}  {N_mc:12.2e}  {N_req_val:12.1e}  {status:>8}  {sigma_improve:11.2f}×')

print()
print('σ improve = gap^(1/4): surface quality improvement factor needed')
print('Thanks to quartic N ∝ σ⁴/λ⁴ scaling, even large gaps require modest improvements.')

---

**To reproduce exact paper results**, change `N_MC = 100_000` in Cell 4 above (~2 min).

For individual module figures, run:
```bash
python src/life_e2e/m1_fiber_coupling.py
python src/life_e2e/m2_throughput_chain.py
# etc.
```